## Double Pendulum with Eureqa Models
**Author**: Dave Heinicke

**Label**: Regression

### Scope

The scope of this notebook is to provide instructions on how to use Eureqa model blueprints with DataRobot. The Data used in this notebook can be found here: 
https://s3.amazonaws.com/datarobot_public_datasets/double-pendulum.csv

The use case is as follows: 
    
    Using a High Speed Camera, log the following:

    t - timestamp
    x1 - Position of the end of upper bar
    x2 - Position of the end of lower bar
    v1 - Velocity of the end of upper bar
    v2 - Velocity of the end of lower bar
    a1 - Acceleration of the end of upper bar
    a2 - Acceleration of the end of lower bar (Target)

We want Eureqa to uncover the underlying relationship:

*a2 = f( t , x1 , x2 , v1 , v2 , a1 )*

### Background

Eureqa leverages an "evolutionary" approach to model creation, testing billions of potential
models per second, and converging on the simplest, most accurate ones that explain your data.

Eureqa makes no prior assumptions about the data set, instead fitting models to the data
dynamically. Compared to other machine learning outputs, Eureqa models are simpler and more
transparent. The models are presented as mathematical equations, so end users can seamlessly
understand results and recommendations.


### Requirements

- Python version 3.7.2
-  DataRobot API version 2.19

Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

#### Import Libraries

In [5]:
import pandas as pd
import datarobot as dr

#### Connect to DataRobot and Define Settings

Connect to DataRobot is mandatory but "define settings" heading should be used especially when datasets are not provided. This way the customer can input his own settings to kickoff the script.

In [7]:
# Start DataRobot Client
dr.Client(token='YOUR_API_KEY',
          endpoint='YOUR_HOSTNAME')

path = 'data/double-pendulum.csv'

# Read in Pendulum data
data = pd.read_csv('data/double-pendulum.csv')
data.head()

,t,x1,x2,v1,v2,a1,a2
0,0.000000,2.36,3.14,-0.0100,-0.01000,-9.24,6.53
1,0.000862,2.36,3.14,-0.0180,-0.00437,-9.24,6.53
2,0.001720,2.36,3.14,-0.0259,0.00126,-9.24,6.53
3,0.002590,2.36,3.14,-0.0339,0.00689,-9.24,6.53
4,0.003450,2.36,3.14,-0.0418,0.01250,-9.24,6.53


#### Create a Project in DataRobot in Manual Mode

This is a regression model. We can just use random partitioning with 5-Fold cross validation (we do not need to keep the pendulum readings in order, we do not need to treat this as a time-aware problem). 

In [9]:
# The Target is "a2", and we can use RMSE as the optmization metric

project = dr.Project.create(data,
                            project_name='Model of a Double Pendulum')

project.set_target(target='a2',
                   metric='RMSE',
                   mode=dr.AUTOPILOT_MODE.MANUAL)

Project(Model of a Double Pendulum)

#### Select a Eureqa Regressor from the Repository

In [11]:
# From the Repository, select a Eureqa Regressor
# List avaialble Eureqa Regressors

eureqa_regressors = [bp for bp in project.get_blueprints() if 'Eureqa Regressor' in bp.model_type]
[print(bp.model_type) for bp in eureqa_regressors]

# Select the one with the default search (3000 generations).
eureqa_bp = eureqa_regressors[0]
eureqa_bp

Eureqa Regressor (Default Search: 3000 Generations)
Eureqa Regressor (Instant Search: 40 Generations)
Eureqa Regressor (Quick Search: 1000 Generations)
Eureqa Regressor (Long-Running Search: 10000 Generations)


Blueprint(Eureqa Regressor (Default Search: 3000 Generations))

In [12]:
# Train the Eureqa Regressor, with default parameters
model_job_id = project.train(eureqa_bp, sample_pct= 64)

dr.models.modeljob.wait_for_async_model_creation(project.id, model_job_id, max_wait=600)

Model('Eureqa Regressor (Default Search: 3000 Generations)')

In [13]:
# Investigate the default model

# Get the default model
default_model = project.get_models()[0]
print(default_model)

# Retrieve the Pareto Front
pf = default_model.get_pareto_front()

# Print the best solution found
default_solutions = pf.solutions
best_solution = [s for s in default_solutions if s.best_model][0]
print(best_solution.expression)

Model('Eureqa Regressor (Default Search: 3000 Generations)')
Target = 18.1051089864661 - 7.14472469444173*x2 - 35.3140912989565*t - 9.89759109372084*min(x1, t) - 1.3224678232697*logistic(v1 - 2.53647862885117 - 598.088169712889*v1*logistic(v2))*if(min(x1, 17.7962969940601 - 6.66794153622069*x2 - 33.6835991897096*t - 9.50649852464752*min(x1, t)), x2, a1)


The default expression isn't what we're looking for.
We need to specify building blocks that match the physical system we're modeling!

First, list all the availalble building blocks to see whats there.

#### Advance Tune the Default Model

In [14]:
# Start an advanced tuning session
tune = default_model.start_advanced_tuning_session()
task_name = tune.get_task_names()[0]

# # List all the eureqa tuning paramaters
param_names = tune.get_parameter_names(task_name)
param_names

['building_block__absolute_value',
 'building_block__addition',
 'building_block__arccosine',
 'building_block__arcsine',
 'building_block__arctangent',
 'building_block__ceiling',
 'building_block__complementary_error_function',
 'building_block__constant',
 'building_block__cosine',
 'building_block__division',
 'building_block__equal-to',
 'building_block__error_function',
 'building_block__exponential',
 'building_block__factorial',
 'building_block__floor',
 'building_block__gaussian_function',
 'building_block__greater-than',
 'building_block__greater-than-or-equal',
 'building_block__hyperbolic_cosine',
 'building_block__hyperbolic_sine',
 'building_block__hyperbolic_tangent',
 'building_block__if-then-else',
 'building_block__input_variable',
 'building_block__integer_constant',
 'building_block__inverse_hyperbolic_cosine',
 'building_block__inverse_hyperbolic_sine',
 'building_block__inverse_hyperbolic_tangent',
 'building_block__less-than',
 'building_block__less-than-or-equa

In [ ]:
# Set the appropriate parameters

tune.description = 'create_physical_model'

# These building blocks don't make sense when modeling a pendulum

params_to_disable = ['building_block__division',
                     'building_block__if-then-else',
                     'building_block__less-than',
                     'building_block__logistic_function',
                     'building_block__maximum',
                     'building_block__minimum',
                     'building_block__natural_logarithm',
                     'building_block__square_root',
                     'building_block__step_function',
                     ]

for p in params_to_disable:
    tune.set_parameter(task_name=task_name,
                       parameter_name = p,
                       value = 'Disabled')

# These building blocks do make sense when modeling a pendulum

params_to_enable = ['building_block__addition',
                     'building_block__constant',
                     'building_block__cosine',
                     'building_block__multiplication',
                     'building_block__sine',
                     'building_block__subtraction'
                     ]

for p in params_to_enable:
    tune.set_parameter(task_name=task_name,
                       parameter_name = p,
                       value = 1)

# Extend the search a bit longer. Longer searches can improve Eureqa models.
tune.set_parameter(task_name=task_name,
                   parameter_name = 'max_generations',
                   value = 4000)
    
job = tune.run()
new_model = job.get_result_when_complete()

In [ ]:
# Retrieve the Pareto Front
pf = new_model.get_pareto_front()

# Print the best solution found
default_solutions = pf.solutions
best_solutions = [s for s in default_solutions][-3:]
[print(s.expression) for s in best_solutions]